In [1]:
from sklearn.metrics import r2_score, make_scorer
from sklearn.linear_model import ElasticNet, HuberRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from scipy.stats import zscore
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
x_test = pd.read_csv('X_test.csv', header=0, index_col=0)
x_train = pd.read_csv('X_train.csv', header=0, index_col=0)
y_train = pd.read_csv('y_train.csv', header=0, index_col=0)

In [3]:
def display(X):
    print(pd.DataFrame(X, columns=x_train.columns))

In [4]:
def preprocessing(X_train, X_test):
    imputer = KNNImputer()
    imputer.fit(X_train)
    return (imputer.transform(X_train), imputer.transform(X_test))

In [5]:
def normalize(X_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    return (scaler.transform(X_train), scaler.transform(X_test))

In [6]:
def outlier_detection(X, y):
    clf = IsolationForest(n_estimators=10, warm_start=True)
    clf.fit(X)  # fit 10 trees  
    clf.set_params(n_estimators=20)  # add 10 more trees  
    clf.fit(X)  # fit the added trees  

    outliers = []

    idx = 0

    for i in clf.predict(X):
        if i == -1:
            outliers.append(idx)
        idx += 1


    return (np.delete(X, outliers, 0), np.delete(y, outliers))
    

In [7]:
def feature_selection(model, X, y):
    raise NotImplementedError

In [8]:
def regression(X_train, y_train, X_test):
    reg = linear_model.Lasso(max_iter=100000)
    reg.fit(X_train, y_train)

    return reg.predict(X_test)

In [9]:
X_processed_train, X_processed_test = preprocessing(x_train, x_test)


In [10]:
X_processed_train, X_processed_test = normalize(X_processed_train, X_processed_test)

In [11]:
X_processed_train, y_processed_train = outlier_detection(X_processed_train, y_train.to_numpy())


In [12]:
results = regression(X_processed_train, y_processed_train, X_processed_test)

In [13]:
asdf = pd.DataFrame(results)
asdf.index.name = 'id'
asdf.rename(columns={ 0: 'y' }, inplace=True)
asdf.to_csv("res.csv")